In [13]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import re
from tensorflow import keras
import time
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, SpatialDropout2D
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Conv2D
from tensorflow.keras.utils import to_categorical

In [2]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)

In [56]:
char_dict = {"char": [], "matrix": []}
char_set = dict()
src = r"../dataset/handwritten_letters"
ignore = ["#", "$", "&", "@"]
for i in range(1000):
    for folder in os.listdir(src):
        if folder in ignore:
            continue
        char_dict["char"].append(folder)
        if folder not in char_set:
            char_set[folder] = sorted_alphanumeric(os.listdir(src + '/' + folder))
        image = tf.keras.preprocessing.image.load_img(f'{src}/{folder}/{char_set[folder][i]}', grayscale = True, color_mode = "grayscale")
        input_arr = keras.preprocessing.image.img_to_array(image)
        input_arr = np.array(input_arr)  # Convert single image to a batch.
        char_dict["matrix"].append(input_arr)

In [57]:
X = np.array(char_dict["matrix"])
y = np.array(char_dict['char'])

In [58]:
X = X.reshape(X.shape[0], 32, 32, 1)

In [59]:
import category_encoders as ce
le =  ce.OneHotEncoder(return_df=False, handle_unknown="ignore")
y = le.fit_transform(y)

C:\Users\sarak\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [60]:
split = int(round(0.9 * len(X)))

train_X = X[:split]/255
train_y = y[:split]
test_X = X[split:]/255
test_y = y[split:]

In [61]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

In [13]:
model = tf.keras.Sequential()
 
model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(32, 32, 1)))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(35, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [77]:
train_y.shape

(15750, 35)

In [14]:
model.fit(train_X, train_y, 
          batch_size=32, epochs=10, verbose=1)

Epoch 1/10
493/493 [==============================] - 6s 8ms/step - loss: 3.4169 - accuracy: 0.0641
Epoch 2/10
493/493 [==============================] - 4s 9ms/step - loss: 2.6338 - accuracy: 0.2452
Epoch 3/10
493/493 [==============================] - 4s 9ms/step - loss: 2.4106 - accuracy: 0.3106
Epoch 4/10
493/493 [==============================] - 4s 9ms/step - loss: 2.2721 - accuracy: 0.3473: 1s - loss: 2.278
Epoch 5/10
493/493 [==============================] - 4s 9ms/step - loss: 2.2093 - accuracy: 0.3588
Epoch 6/10
493/493 [==============================] - 5s 10ms/step - loss: 2.1341 - accuracy: 0.3766
Epoch 7/10
493/493 [==============================] - 4s 9ms/step - loss: 2.0694 - accuracy: 0.3901
Epoch 8/10
493/493 [==============================] - 4s 8ms/step - loss: 2.0400 - accuracy: 0.3892
Epoch 9/10
493/493 [==============================] - 4s 8ms/step - loss: 1.9893 - accuracy: 0.4107
Epoch 10/10
493/493 [==============================] - 4s 9ms/step - loss: 1.9891

In [93]:
pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')    

Accuracy on training data: 0.5407618880271912% 
 Error on test data: 0.45923811197280884

Accuracy on test data: 0.4617142975330353% 
 Error on test data: 0.5382857024669647


### Test 2 - 25 epochs

In [15]:
model.fit(train_X, train_y, 
          batch_size=32, epochs=25, verbose=1)

Epoch 1/25
493/493 [==============================] - 5s 10ms/step - loss: 1.9511 - accuracy: 0.4204
Epoch 2/25
493/493 [==============================] - 5s 11ms/step - loss: 1.9156 - accuracy: 0.4297
Epoch 3/25
493/493 [==============================] - 5s 10ms/step - loss: 1.9190 - accuracy: 0.4301
Epoch 4/25
493/493 [==============================] - 5s 9ms/step - loss: 1.9171 - accuracy: 0.4286
Epoch 5/25
493/493 [==============================] - 5s 9ms/step - loss: 1.8909 - accuracy: 0.4312
Epoch 6/25
493/493 [==============================] - 5s 10ms/step - loss: 1.8864 - accuracy: 0.4380
Epoch 7/25
493/493 [==============================] - 4s 9ms/step - loss: 1.8646 - accuracy: 0.4422
Epoch 8/25
493/493 [==============================] - 5s 10ms/step - loss: 1.8519 - accuracy: 0.4448
Epoch 9/25
493/493 [==============================] - 4s 9ms/step - loss: 1.8504 - accuracy: 0.4404
Epoch 10/25
493/493 [==============================] - 4s 8ms/step - loss: 1.8468 - accuracy: 0

In [16]:
pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')    

Accuracy on training data: 0.5999364852905273% 
 Error on test data: 0.40006351470947266

Accuracy on test data: 0.4811428487300873% 
 Error on test data: 0.5188571512699127


### Test 3 - trying other layers 84,4%

In [20]:
model = tf.keras.Sequential()
 
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(35, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [21]:
model.fit(train_X, train_y, 
          batch_size=32, epochs=25, verbose=1)

pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')    

Epoch 1/25
493/493 [==============================] - 39s 76ms/step - loss: 1.8560 - accuracy: 0.4953
Epoch 2/25
493/493 [==============================] - 34s 69ms/step - loss: 0.4103 - accuracy: 0.8704
Epoch 3/25
493/493 [==============================] - 32s 64ms/step - loss: 0.2926 - accuracy: 0.90321s - loss: 0.2929 - accuracy
Epoch 4/25
493/493 [==============================] - 31s 63ms/step - loss: 0.2173 - accuracy: 0.9292
Epoch 5/25
493/493 [==============================] - 33s 66ms/step - loss: 0.1710 - accuracy: 0.9413
Epoch 6/25
493/493 [==============================] - 34s 69ms/step - loss: 0.1317 - accuracy: 0.9533
Epoch 7/25
493/493 [==============================] - 34s 68ms/step - loss: 0.1251 - accuracy: 0.9592
Epoch 8/25
493/493 [==============================] - 34s 70ms/step - loss: 0.0964 - accuracy: 0.9683
Epoch 9/25
493/493 [==============================] - 33s 68ms/step - loss: 0.0672 - accuracy: 0.9756
Epoch 10/25
493/493 [==============================] -

### Test 4 - removing one dense layer 85,2%

In [22]:
model = tf.keras.Sequential()
 
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(35, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
model.fit(train_X, train_y, 
          batch_size=32, epochs=25, verbose=1)

pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')    

Epoch 1/25
493/493 [==============================] - 34s 66ms/step - loss: 1.7600 - accuracy: 0.5199
Epoch 2/25
493/493 [==============================] - 42s 84ms/step - loss: 0.4430 - accuracy: 0.8685
Epoch 3/25
493/493 [==============================] - 34s 69ms/step - loss: 0.2980 - accuracy: 0.9065
Epoch 4/25
493/493 [==============================] - 35s 71ms/step - loss: 0.2292 - accuracy: 0.92590s - loss: 0.2292 - accu
Epoch 5/25
493/493 [==============================] - 37s 74ms/step - loss: 0.1897 - accuracy: 0.9382
Epoch 6/25
493/493 [==============================] - 38s 76ms/step - loss: 0.1367 - accuracy: 0.9522
Epoch 7/25
493/493 [==============================] - 35s 71ms/step - loss: 0.1023 - accuracy: 0.96420s - loss: 0.1021 - accuracy
Epoch 8/25
493/493 [==============================] - 34s 69ms/step - loss: 0.0880 - accuracy: 0.9715
Epoch 9/25
493/493 [==============================] - 45s 91ms/step - loss: 0.0671 - accuracy: 0.9745
Epoch 10/25
493/493 [=========

### Test 5 - trying dropout layer 86,7%

In [24]:
model = tf.keras.Sequential()
 
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(35, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [25]:
model.fit(train_X, train_y, 
          batch_size=32, epochs=25, verbose=1)

pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')    

Epoch 1/25
493/493 [==============================] - 38s 73ms/step - loss: 2.0143 - accuracy: 0.4469
Epoch 2/25
493/493 [==============================] - 35s 72ms/step - loss: 0.6042 - accuracy: 0.8143
Epoch 3/25
493/493 [==============================] - 35s 71ms/step - loss: 0.4330 - accuracy: 0.8645
Epoch 4/25
493/493 [==============================] - 35s 71ms/step - loss: 0.3499 - accuracy: 0.8878
Epoch 5/25
493/493 [==============================] - 36s 73ms/step - loss: 0.3218 - accuracy: 0.8967
Epoch 6/25
493/493 [==============================] - 36s 73ms/step - loss: 0.2710 - accuracy: 0.9135
Epoch 7/25
493/493 [==============================] - 35s 72ms/step - loss: 0.2453 - accuracy: 0.9184
Epoch 8/25
493/493 [==============================] - 35s 71ms/step - loss: 0.2199 - accuracy: 0.9259
Epoch 9/25
493/493 [==============================] - 35s 71ms/step - loss: 0.2165 - accuracy: 0.9262
Epoch 10/25
493/493 [==============================] - 34s 70ms/step - loss: 0.189

### Test 6 - adding another dropout layer 86,1%

In [27]:
model = tf.keras.Sequential()
 
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(35, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [28]:
model.fit(train_X, train_y, 
          batch_size=32, epochs=25, verbose=1)

pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')    

Epoch 1/25
493/493 [==============================] - 36s 66ms/step - loss: 2.6413 - accuracy: 0.2752
Epoch 2/25
493/493 [==============================] - 31s 64ms/step - loss: 1.0117 - accuracy: 0.6993
Epoch 3/25
493/493 [==============================] - 32s 65ms/step - loss: 0.7851 - accuracy: 0.7575
Epoch 4/25
493/493 [==============================] - 32s 65ms/step - loss: 0.6623 - accuracy: 0.79470s - loss: 0.6624 - 
Epoch 5/25
493/493 [==============================] - 32s 65ms/step - loss: 0.6031 - accuracy: 0.8130
Epoch 6/25
493/493 [==============================] - 32s 64ms/step - loss: 0.5832 - accuracy: 0.8172
Epoch 7/25
493/493 [==============================] - 33s 66ms/step - loss: 0.5262 - accuracy: 0.8339
Epoch 8/25
493/493 [==============================] - 34s 69ms/step - loss: 0.4992 - accuracy: 0.8442
Epoch 9/25
493/493 [==============================] - 36s 73ms/step - loss: 0.4601 - accuracy: 0.8517
Epoch 10/25
493/493 [==============================] - 36s 73m

### Test 7 - adding another Conv2D and MaxPooling2D 86,9%

In [29]:
model = tf.keras.Sequential()
 
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(35, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
model.fit(train_X, train_y, 
          batch_size=32, epochs=25, verbose=1)

pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')    

Epoch 1/25
493/493 [==============================] - 39s 74ms/step - loss: 2.7882 - accuracy: 0.2270
Epoch 2/25
493/493 [==============================] - 36s 73ms/step - loss: 1.1217 - accuracy: 0.6563
Epoch 3/25
493/493 [==============================] - 34s 69ms/step - loss: 0.8634 - accuracy: 0.7347
Epoch 4/25
493/493 [==============================] - 35s 71ms/step - loss: 0.7485 - accuracy: 0.7644
Epoch 5/25
493/493 [==============================] - 39s 80ms/step - loss: 0.6763 - accuracy: 0.7879
Epoch 6/25
493/493 [==============================] - 41s 84ms/step - loss: 0.6131 - accuracy: 0.80561s - loss: 0
Epoch 7/25
493/493 [==============================] - 35s 71ms/step - loss: 0.5876 - accuracy: 0.8154
Epoch 8/25
493/493 [==============================] - 34s 69ms/step - loss: 0.5508 - accuracy: 0.81960s - los
Epoch 9/25
493/493 [==============================] - 36s 72ms/step - loss: 0.5173 - accuracy: 0.8350
Epoch 10/25
493/493 [==============================] - 43s 87m

### Test 8 - adding two smaller Dense layers 85%

In [31]:
model = tf.keras.Sequential()
 
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
 
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(35, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [32]:
model.fit(train_X, train_y, 
          batch_size=32, epochs=25, verbose=1)

pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')

Epoch 1/25
493/493 [==============================] - 40s 77ms/step - loss: 2.9688 - accuracy: 0.1761
Epoch 2/25
493/493 [==============================] - 44s 89ms/step - loss: 1.2583 - accuracy: 0.6123
Epoch 3/25
493/493 [==============================] - 35s 72ms/step - loss: 0.9903 - accuracy: 0.6978
Epoch 4/25
493/493 [==============================] - 41s 83ms/step - loss: 0.8421 - accuracy: 0.7318
Epoch 5/25
493/493 [==============================] - 49s 99ms/step - loss: 0.7736 - accuracy: 0.7615
Epoch 6/25
493/493 [==============================] - 65s 132ms/step - loss: 0.7065 - accuracy: 0.7819
Epoch 7/25
493/493 [==============================] - 41s 83ms/step - loss: 0.6422 - accuracy: 0.7985
Epoch 8/25
493/493 [==============================] - 46s 94ms/step - loss: 0.6256 - accuracy: 0.7984
Epoch 9/25
493/493 [==============================] - 34s 69ms/step - loss: 0.5799 - accuracy: 0.8163
Epoch 10/25
493/493 [==============================] - 29s 60ms/step - loss: 0.56

### Test 9 - Spatial Dopout 86,6%

In [20]:
model = tf.keras.Sequential()
 
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(SpatialDropout2D(0.5))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(35, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [21]:
model.fit(train_X, train_y, 
          batch_size=32, epochs=25, verbose=1)

pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')

Epoch 1/25
493/493 [==============================] - 32s 61ms/step - loss: 2.7342 - accuracy: 0.25010s - loss: 2.7447 - accuracy
Epoch 2/25
493/493 [==============================] - 24s 49ms/step - loss: 1.0573 - accuracy: 0.6865
Epoch 3/25
493/493 [==============================] - 24s 49ms/step - loss: 0.7912 - accuracy: 0.7550
Epoch 4/25
493/493 [==============================] - 25s 51ms/step - loss: 0.6888 - accuracy: 0.7917
Epoch 5/25
493/493 [==============================] - 26s 52ms/step - loss: 0.6352 - accuracy: 0.8040
Epoch 6/25
493/493 [==============================] - 27s 54ms/step - loss: 0.5564 - accuracy: 0.8227
Epoch 7/25
493/493 [==============================] - 27s 54ms/step - loss: 0.5385 - accuracy: 0.8322
Epoch 8/25
493/493 [==============================] - 26s 52ms/step - loss: 0.5109 - accuracy: 0.8450
Epoch 9/25
493/493 [==============================] - 26s 54ms/step - loss: 0.4648 - accuracy: 0.8554
Epoch 10/25
493/493 [==============================] -

### Test 10 - no preprocessing 85,7%

In [39]:
model = tf.keras.Sequential()
 
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(35, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [42]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 4, 4, 32)          9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 2, 2, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 2, 2, 32)         

In [40]:
model.fit(train_X, train_y, 
          batch_size=32, epochs=25, verbose=1)

pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')

Epoch 1/25
493/493 [==============================] - 53s 101ms/step - loss: 2.8054 - accuracy: 0.2320
Epoch 2/25
493/493 [==============================] - 46s 94ms/step - loss: 1.0758 - accuracy: 0.6775
Epoch 3/25
493/493 [==============================] - 51s 103ms/step - loss: 0.8178 - accuracy: 0.7501
Epoch 4/25
493/493 [==============================] - 63s 129ms/step - loss: 0.7251 - accuracy: 0.7738
Epoch 5/25
493/493 [==============================] - 75s 153ms/step - loss: 0.6463 - accuracy: 0.7970
Epoch 6/25
493/493 [==============================] - 72s 146ms/step - loss: 0.6035 - accuracy: 0.8060
Epoch 7/25
493/493 [==============================] - 37s 75ms/step - loss: 0.5580 - accuracy: 0.8254
Epoch 8/25
493/493 [==============================] - 36s 74ms/step - loss: 0.5450 - accuracy: 0.8277
Epoch 9/25
493/493 [==============================] - 37s 74ms/step - loss: 0.5185 - accuracy: 0.8391
Epoch 10/25
493/493 [==============================] - 36s 74ms/step - loss: 

### Test 11 - loading image with Keras, 1000 images 94,1%

In [62]:
model = tf.keras.Sequential()
 
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(35, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [63]:
model.fit(train_X, train_y, 
          batch_size=32, epochs=25, verbose=1)

pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')

Epoch 1/25
985/985 [==============================] - 73s 73ms/step - loss: 2.3509 - accuracy: 0.3406
Epoch 2/25
985/985 [==============================] - 77s 79ms/step - loss: 0.8851 - accuracy: 0.7295
Epoch 3/25
985/985 [==============================] - 76s 77ms/step - loss: 0.7400 - accuracy: 0.7710
Epoch 4/25
985/985 [==============================] - 81s 82ms/step - loss: 0.6410 - accuracy: 0.7975
Epoch 5/25
985/985 [==============================] - 79s 80ms/step - loss: 0.5916 - accuracy: 0.8147
Epoch 6/25
985/985 [==============================] - 76s 77ms/step - loss: 0.5447 - accuracy: 0.8287
Epoch 7/25
985/985 [==============================] - 79s 80ms/step - loss: 0.5190 - accuracy: 0.8339
Epoch 8/25
985/985 [==============================] - 77s 78ms/step - loss: 0.4844 - accuracy: 0.8455
Epoch 9/25
985/985 [==============================] - 78s 80ms/step - loss: 0.4600 - accuracy: 0.8533
Epoch 10/25
985/985 [==============================] - 76s 77ms/step - loss: 0.445